# Assignment #3: A simple language classifier with scikit-learn and PyTorch

Author: Pierre Nugues

## Objectives

In this assignment, you will implement a language detector inspired and simplified from Google's _Compact language detector_, version 3 (CLD3): https://github.com/google/cld3. CLD3 is written in C++ and its code is available from GitHub. The objectives of the assignment are to:
* Write a program to classify languages
* Use neural networks with sklearn and PyTorch
* Know what a classifier is
* Write a short report of 1 to 2 pages to describe your program. You will notably comment the performance you obtained and how you could improve it.

## Description

### System Overview

Read the GitHub description of CLD3, https://github.com/google/cld3, (_Model_ section). In your individual report you will:
1. Summarize the system in two or three sentences;
2. Outline the CLD3 overall architecture in a figure. Use building blocks only and do not specify the parameters.

## Imports

In [35]:
import bz2
import json
import os
import numpy as np
import requests
import sys
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, classification_report
from sklearn.metrics import confusion_matrix
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import hashlib
from tqdm import tqdm
import regex as re

In [11]:
random.seed(1234)
np.random.seed(1234)
torch.manual_seed(1234)

## Dataset

As dataset, we will use Tatoeba, https://tatoeba.org/eng/downloads. It consists of more than 8 million short texts in 347 languages and it is available in one file called `sentences.csv`.

The dataset is structured this way: There is one text per line, where each line consists of the three following fields separated by tabulations and ended by a carriage return:
```
sentence id [tab] language code [tab] text [cr]
```
Each text (sentence) has a unique id and has a language code that follows the ISO 639-3 standard (see below). 

### Scope of the lab

In this lab, you will consider six languages only: French (fra), Japanese (jpn), Chinese (cmn), English (eng), Swedish (swe), and Danish (dan). Below is an excerpt of the Tatoeba dataset limited to three languages: 

```
1276    eng     Let's try something.
1277    eng     I have to go to sleep.
1280    eng     Today is June 18th and it is Muiriel's birthday!
...
1115    fra     Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.
1279    fra     Je ne supporte pas ce type.
1441    fra     Pour une fois dans ma vie je fais un bon geste... Et ça ne sert à rien.
...
337413  swe     Vi trodde att det var ett flygande tefat.
341910  swe     Detta är huset jag bodde i när jag var barn.
341938  swe     Vi hade roligt på stranden igår.
...
```
Tatoeba is updated continuously. The examples from this dataset come from a corpus your instructor downloaded on September 23, 2021.

### Understanding the ${X}$ matrix (feature matrix)

You will now investigate the CLD3 features:
 *  What are the features CLD3 extracts from each text?
 * Create manually a simplified ${X}$ matrix where you will represent the 9 texts with CLD3 features. You will use a restricted set of features: You will only consider the letters _a_, _b_, and _n_ and the bigrams _an_, _ba_, and _na_. You will ignore the the rest of letters and bigrams as well as the trigrams. Your matrix will have 9 rows and 6 columns, each column will contain these counts: `[#a, #b, #n, #an, #ba, #na]`.

The CLD3's original description uses relative frequencies (counts of a letter divided by the total counts of letters in the text). Here, you will use the raw counts. To help you, your instructor filled the fourth row of the matrix corresponding to the first text in French. Fill in the rest. You will __include this matrix in your report__. 

$\mathbf{X} =
\begin{bmatrix}
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
8& 0& 8& 1&0&0\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
-& -& -& -&-& -\\
\end{bmatrix}$
; $\mathbf{y} =
\begin{bmatrix}
     \text{eng} \\
     \text{eng}\\
     \text{eng}\\
    \text{fra}\\
   \text{fra}  \\
     \text{fra}\\
    \text{swe}\\
 \text{swe}   \\
 \text{swe}   
\end{bmatrix}$

To help you check your counts, you can use the `str.count()` function

In [16]:
example_ngrams = ['a', 'b', 'n', 'an', 'ba', 'na']

In [17]:
"""Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.""".count('a')

8

In [18]:
"""Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent.""".count('an')

1

In [19]:
my_string = """Lorsqu'il a demandé qui avait cassé la fenêtre, tous les garçons ont pris un air innocent."""
row = []
for ngram in example_ngrams:
    row += [my_string.count(ngram)]
row

[8, 0, 8, 1, 0, 0]

## Getting the Dataset

Before you start programming, download the Tatoeba dataset. You can use the instructions:

In [7]:
!wget https://downloads.tatoeba.org/exports/sentences.tar.bz2

--2023-09-21 13:52:16--  https://downloads.tatoeba.org/exports/sentences.tar.bz2
Slår upp downloads.tatoeba.org (downloads.tatoeba.org)... 94.130.77.194
Ansluter till downloads.tatoeba.org (downloads.tatoeba.org)|94.130.77.194|:443 … ansluten.
HTTP-begäran skickad, väntar på svar... 200 OK
Längd: 183274899 (175M) [application/octet-stream]
Sparar till: ”sentences.tar.bz2”

sentences.tar.bz2   100%[===================>] 174,78M  17,8MB/s    om 9,7s    

2023-09-21 13:52:26 (18,0 MB/s) - ”sentences.tar.bz2” sparades [183274899/183274899]



In [8]:
!tar -xvjf sentences.tar.bz2

x sentences.csv


### Loading the Dataset

Run the code to read the dataset and split it into lines. You may have to change the path

In [20]:
dataset_large = open('sentences.csv', encoding='utf8').read().strip()
dataset_large = dataset_large.split('\n')
dataset_large[:10]

['1\tcmn\t我們試試看！',
 '2\tcmn\t我该去睡觉了。',
 '3\tcmn\t你在干什麼啊？',
 '4\tcmn\t這是什麼啊？',
 '5\tcmn\t今天是６月１８号，也是Muiriel的生日！',
 '6\tcmn\t生日快乐，Muiriel！',
 '7\tcmn\tMuiriel现在20岁了。',
 '8\tcmn\t密码是"Muiriel"。',
 '9\tcmn\t我很快就會回來。',
 '10\tcmn\t我不知道。']

The size may vary as new documents are added every day to _Tatoeba_

In [21]:
len(dataset_large)

11611245

Run the code to split the fields and remove possible whitespaces

In [22]:
dataset_large = list(map(lambda x: tuple(x.split('\t')), dataset_large))
dataset_large = list(map(lambda x: tuple(map(str.strip, x)), dataset_large))
dataset_large[:3]

[('1', 'cmn', '我們試試看！'), ('2', 'cmn', '我该去睡觉了。'), ('3', 'cmn', '你在干什麼啊？')]

In [23]:
from collections import Counter
counter = Counter(map(lambda x: x[1], dataset_large))

Again the figures may vary

In [24]:
counter.most_common(30)

[('eng', 1829233),
 ('rus', 1013772),
 ('ita', 863056),
 ('tur', 728016),
 ('epo', 727449),
 ('kab', 686241),
 ('ber', 649317),
 ('deu', 626486),
 ('fra', 568043),
 ('por', 422018),
 ('spa', 402305),
 ('hun', 394468),
 ('jpn', 238488),
 ('heb', 200838),
 ('ukr', 184377),
 ('nld', 176953),
 ('fin', 146641),
 ('pol', 122378),
 ('lit', 93912),
 ('mkd', 78095),
 ('tgl', 74892),
 ('cmn', 73632),
 ('ces', 73545),
 ('mar', 73277),
 ('ara', 62802),
 ('dan', 58848),
 ('tok', 53870),
 ('swe', 52960),
 ('lat', 49153),
 ('srp', 47878)]

## Restricting the Dataset to a few Languages 

The Tatoeba dataset is very large. You will first extract a subset of it

Write the code to extract texts in the languages below. For each language, you limit the number of documents to 50,000 or less if the language has less documents.
You will call the resulting dataset: `dataset`

The languages

In [7]:
langs = ['fra', 'cmn', 'jpn', 'eng', 'swe', 'dan']

The maximal number of documents per language

In [8]:
MAX_DOCS = 50000

Write a loop that:
1. Extracts a list of all the documents in a certain language from the dataset
2. Shuffles this list with `random.shuffle()`
3. Adds `MAX_DOCS` to `dataset`. You just need to use a slice

In [29]:
# Write your code here

dataset = []
for lang in langs:
    languageset = []
    languageset += list(filter(lambda x: x[1] == lang, dataset_large))
    random.shuffle(languageset)
    languageset = languageset[:MAX_DOCS]
    dataset += languageset


In [30]:
random.shuffle(dataset)

In [31]:
len(dataset)

300000

In [32]:
dataset[:5]

[('3912898', 'eng', "I don't like her so much."),
 ('3065609', 'cmn', '你因该请一天的假期。'),
 ('3741336', 'dan', 'Du blev snydt.'),
 ('11784764', 'dan', 'Tom har absolut gehør.'),
 ('9711742', 'eng', 'Tom gave the hungry boy something to eat.')]

## Utilities

Before you can use the dataset to train a model, you need to convert it into numbers. You will carry this with out the following steps and you will write a corresponding function.
1. You will extract the $n$-grams up to trigrams (`all_ngrams()`);
2. Trigrams can create many symbols that most student's machines cannot process. You will reduce their numbers using hash codes (`hash_ngrams()`);
3. You will compute the relative frequencies of the $n$-grams, replaced here by the hash codes (`calc_ref_freq()`).
4. The results will be stored in three dictionaries, for characters, bigrams, and trigrams. You will merge these dictionaries into one (`shift_keys()`).

You will then apply the functions to vectorize the dataset.

### Extracting $n$-grams
The goal of this section is that you extract the $n$-grams from a text. By default, you will lowercase the text. The result will have the form: `[chars, bigrams, trigrams]`

Write a function to extract the $n$-grams of a sentence: `ngrams(sentence, n=1, lc=True)`, `n` is a parameters. You can use list slices for this.

In [68]:

def ngrams(sentence, n=1, lc=True):
    ngrams = [sentence[idx:idx + n] for idx in range(len(sentence) - n + 1)]
    return ngrams

In [69]:
ngrams('try something.')

['t', 'r', 'y', ' ', 's', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', '.']

In [70]:
ngrams('try something.', n=2)

['tr', 'ry', 'y ', ' s', 'so', 'om', 'me', 'et', 'th', 'hi', 'in', 'ng', 'g.']

We now use this function to extract all the $n$-grams

In [71]:
def all_ngrams(sentence, max_ngram=3, lc=True):
    all_ngram_list = []
    for i in range(1, max_ngram + 1):
        all_ngram_list += [ngrams(sentence, n=i, lc=lc)]
    return all_ngram_list

In [72]:
all_ngrams('try something.')

[['t', 'r', 'y', ' ', 's', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', '.'],
 ['tr',
  'ry',
  'y ',
  ' s',
  'so',
  'om',
  'me',
  'et',
  'th',
  'hi',
  'in',
  'ng',
  'g.'],
 ['try',
  'ry ',
  'y s',
  ' so',
  'som',
  'ome',
  'met',
  'eth',
  'thi',
  'hin',
  'ing',
  'ng.']]

### Hashing

We consider languages with many characters that will make the number of bigrams and trigrams impossible to process. We will use the _hashing trick_ to reduce them, where we will gather $n$-grams into subsets using hash codes.

Each item will have this format:
`[char_hcodes, bigram_hcodes, trigram_hcodes]`.

#### Description

Python has a built-in hashing function that returns a unique numerical signature for a given string

In [81]:
hash('a'), hash('ab'), hash('abc')

(-522534217003743531, 3209738916746912879, 3610042094769522055)

If we take the remainder (modulo) of a division by 5, we reduce the possible codes to: 0, 1, 2, 3, or 4

In [83]:
list(map(lambda x: x % 5, (hash('a'), hash('ab'), hash('abc'))))

[4, 4, 0]

#### Implementation

We set maximal numbers for our $n$-grams using these divisors

In [89]:
MAX_CHARS = 521
MAX_BIGRAMS = 1031
MAX_TRIGRAMS = 1031

Here strings have integer codes within the range [0, `MAX_CHARS`[

In [90]:
list(map(lambda x: x % MAX_CHARS, (hash('a'), hash('ab'), hash('abc'))))

[353, 342, 295]

Hash codes may vary across machines and Marcus Klang wrote this function to have reproducible codes

In [91]:
def reproducible_hash(string):
    """
    reproducible hash on any string
    
    Arguments:
       string: python string object
    
    Returns:
       signed int64
    """
    
    # We are using MD5 for speed not security.
    h = hashlib.md5(string.encode("utf-8"), usedforsecurity=False)
    return int.from_bytes(h.digest()[0:8], 'big', signed=True)

In [93]:
reproducible_hash('a')

919145239626757800

In [94]:
reproducible_hash('a') % MAX_CHARS

234

### Converting $n$-grams to hash codes
You will now convert the $n$-grams to hash codes


In [92]:
MAXES = [MAX_CHARS, MAX_BIGRAMS, MAX_TRIGRAMS]

Create a `hash_ngrams` function that creates a list of hash codes from a list of $n$-grams. As arguments, you will have the list of $n$-grams `[chars, bigrams, trigrams]` as well as the list of dividers (`MAXES`).

The output format will be a list of three lists:

`[char_hcodes, bigram_hcodes, trigram_hcodes]`.

In [174]:
# Write your code
def hash_ngrams(ngrams, modulos):
    hash_codes = [[] for _ in range(len(ngrams))]
    for i in range(len(ngrams)):
        for ngram in ngrams[i]:
            hash_codes[i].append(reproducible_hash(ngram) % modulos[i])
    return hash_codes

In [ ]:
all_ngrams('try something.')

In [175]:
hash_ngrams(all_ngrams('try something.'), MAXES)

[[432, 437, 309, 86, 331, 97, 100, 32, 432, 332, 233, 310, 31, 442],
 [6, 765, 224, 203, 557, 176, 590, 711, 527, 757, 919, 57, 685],
 [848, 617, 468, 456, 873, 996, 287, 10, 817, 674, 960, 399]]

### Functions to Count Hash Codes

Write a function `calc_rel_freq(codes)` to count the codes. As in CLD3, you will return the relative frequencies.

This is just an application of `Counter` to a list of codes and then a division by the length.

The input is a list of codes and the output is a `Counter` object of relative frequencies.

In [178]:
# Write your code
def calc_rel_freq(codes):
    frequencies = {}
    for code in codes :
        if code in frequencies:
            frequencies[code] += 1
        else:
            frequencies[code] = 1
    for key in frequencies:
        frequencies[key] = frequencies[key]/len(codes)
    cnt = Counter(frequencies)
    return cnt

In [179]:
hash_ngrams(all_ngrams('try something.'), MAXES)

[[432, 437, 309, 86, 331, 97, 100, 32, 432, 332, 233, 310, 31, 442],
 [6, 765, 224, 203, 557, 176, 590, 711, 527, 757, 919, 57, 685],
 [848, 617, 468, 456, 873, 996, 287, 10, 817, 674, 960, 399]]

In [181]:
list(map(calc_rel_freq, hash_ngrams(all_ngrams('try something.'), MAXES)))

[Counter({432: 0.14285714285714285,
          437: 0.07142857142857142,
          309: 0.07142857142857142,
          86: 0.07142857142857142,
          331: 0.07142857142857142,
          97: 0.07142857142857142,
          100: 0.07142857142857142,
          32: 0.07142857142857142,
          332: 0.07142857142857142,
          233: 0.07142857142857142,
          310: 0.07142857142857142,
          31: 0.07142857142857142,
          442: 0.07142857142857142}),
 Counter({6: 0.07692307692307693,
          765: 0.07692307692307693,
          224: 0.07692307692307693,
          203: 0.07692307692307693,
          557: 0.07692307692307693,
          176: 0.07692307692307693,
          590: 0.07692307692307693,
          711: 0.07692307692307693,
          527: 0.07692307692307693,
          757: 0.07692307692307693,
          919: 0.07692307692307693,
          57: 0.07692307692307693,
          685: 0.07692307692307693}),
 Counter({848: 0.08333333333333333,
          617: 0.08333333333333

### Merge the Dictionaries

In the results above, we have three counter objects with numerical keys (the hash codes). You will build one dictionary of them.

There is a key overlap and we must take care that a same hash code for the unigrams is not the same as in the bigrams. We will then shift the keys.

The keys range from:
1. Unigrams from 0 to 521, [0, MAX_CHARS[
2. Bigrams from 0 to 1031, [0, MAX_BIGRAMS[
3. Trigrams from 1 to 1031, [0, MAX_TRIGRAMS[

You will leave the unigrams keys as they are. You will shift the bigram keys by MAX_CHARS, and the trigram keys by MAX_CHARS + MAX_BIGRAMS. You can reuse the code below

In [182]:
MAX_SHIFT = []
for i in range(len(MAXES)):
    MAX_SHIFT += [sum(MAXES[:i])]

In [183]:
MAX_SHIFT

[0, 521, 1552]

Write a `shift_keys(dicts, MAX_SHIFT)` function that takes a list of dictionaries as input and the list of shifts and that a new unique dictionary, where the numerical keys have been shifted by the numbers in `MAX_SHIFT`

In [233]:
# Write your code here
def shift_keys(dicts, MAX_SHIFT):
    new_dict = {}
    list_dicts = list(dicts)
    
    for i in range(len(MAX_SHIFT)):
        for key in list_dicts[i]:
            new_dict[key + MAX_SHIFT[i]] = list_dicts[i][key]

    return new_dict

In [ ]:
list(map(calc_rel_freq, hash_ngrams(all_ngrams('try something.'), MAXES)))

In [ ]:
shift_keys(map(calc_rel_freq, hash_ngrams(all_ngrams('try something.'), MAXES)), MAX_SHIFT)

Finally, we assemble all these utilities in a function

In [235]:
def build_freq_dict(sentence, MAXES=MAXES, MAX_SHIFT=MAX_SHIFT):
    hngrams = hash_ngrams(all_ngrams(sentence), MAXES)
    fhcodes = map(calc_rel_freq, hngrams)
    return shift_keys(fhcodes, MAX_SHIFT)

In [236]:
build_freq_dict('try something.')

{432: 0.14285714285714285,
 437: 0.07142857142857142,
 309: 0.07142857142857142,
 86: 0.07142857142857142,
 331: 0.07142857142857142,
 97: 0.07142857142857142,
 100: 0.07142857142857142,
 32: 0.07142857142857142,
 332: 0.07142857142857142,
 233: 0.07142857142857142,
 310: 0.07142857142857142,
 31: 0.07142857142857142,
 442: 0.07142857142857142,
 527: 0.07692307692307693,
 1286: 0.07692307692307693,
 745: 0.07692307692307693,
 724: 0.07692307692307693,
 1078: 0.07692307692307693,
 697: 0.07692307692307693,
 1111: 0.07692307692307693,
 1232: 0.07692307692307693,
 1048: 0.07692307692307693,
 1278: 0.07692307692307693,
 1440: 0.07692307692307693,
 578: 0.07692307692307693,
 1206: 0.07692307692307693,
 2400: 0.08333333333333333,
 2169: 0.08333333333333333,
 2020: 0.08333333333333333,
 2008: 0.08333333333333333,
 2425: 0.08333333333333333,
 2548: 0.08333333333333333,
 1839: 0.08333333333333333,
 1562: 0.08333333333333333,
 2369: 0.08333333333333333,
 2226: 0.08333333333333333,
 2512: 0.08333

## Converting the Dataset
We can now enrich the dataset with a numerical representation of the sentence. We use the utility functions and we call this new version: `dataset_num`

In [237]:
dataset[:2]

[('3912898', 'eng', "I don't like her so much."),
 ('3065609', 'cmn', '你因该请一天的假期。')]

In [238]:
dataset_num = []
for datapoint in tqdm(dataset):
    dataset_num += [list(datapoint) + [build_freq_dict(datapoint[2])]]

100%|██████████| 300000/300000 [00:43<00:00, 6896.52it/s] 


In [243]:
dataset_num[:2]

[['3912898',
  'eng',
  "I don't like her so much.",
  {130: 0.04,
   86: 0.2,
   371: 0.04,
   97: 0.08,
   310: 0.04,
   323: 0.04,
   432: 0.04,
   15: 0.04,
   233: 0.04,
   514: 0.04,
   32: 0.08,
   332: 0.08,
   437: 0.04,
   331: 0.04,
   100: 0.04,
   69: 0.04,
   196: 0.04,
   442: 0.04,
   1100: 0.041666666666666664,
   677: 0.041666666666666664,
   1081: 0.041666666666666664,
   1440: 0.041666666666666664,
   846: 0.041666666666666664,
   778: 0.041666666666666664,
   1492: 0.041666666666666664,
   900: 0.041666666666666664,
   909: 0.041666666666666664,
   735: 0.041666666666666664,
   1070: 0.041666666666666664,
   995: 0.041666666666666664,
   1027: 0.041666666666666664,
   806: 0.041666666666666664,
   1066: 0.08333333333333333,
   555: 0.041666666666666664,
   724: 0.041666666666666664,
   1078: 0.041666666666666664,
   795: 0.041666666666666664,
   1466: 0.041666666666666664,
   857: 0.041666666666666664,
   717: 0.041666666666666664,
   1253: 0.041666666666666664,
  

## Programming: Building ${X}$

You will now build the ${X}$ matrix.

### Vectorizing the features

The CLD3 architecture uses embeddings. In this lab, we will simplify it and we will use a feature vector instead consisting of the character frequencies. For example, you will represent the text:

`"Let's try something."`

with:

`{'l': 0.05, 'e': 0.1, 't': 0.15, "'": 0.05, 's': 0.1, ' ': 0.1, 
 'r': 0.05, 'y': 0.05, 'o': 0.05, 'm': 0.05, 'h': 0.05, 'i': 0.05, 
 'n': 0.05, 'g': 0.05, '.': 0.05}`

Note that we used characters and not codes to make it more legible.

To create the ${X}$ matrix, we need to transform the dictionaries of `dataset_num` into numerical vectors. The `DictVectorizer` class from the scikit-learn library, see here [https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html], has two methods, `fit()` and `transform()`, and a combination of both `fit_transform()` to convert dictionaries into such vectors.

You will now write the code to:

1. Extract the hash code frequency dictionaries from `dataset_num` corresponding to its 3rd index;
2. Convert the list of dictionaries into an ${X}$ matrix using `DictVectorizer`.

#### Extracting the character frequencies

Produce a new list of datapoints with the $n$-grams. Each item in this list will be a dictionary. You will call it `X_cat`

In [330]:
# Write your code here
X_cat = []
for i in range(len(dataset_num)):
    X_cat += [dataset_num[i][3]]


In [331]:
X_cat[0]

{130: 0.04,
 86: 0.2,
 371: 0.04,
 97: 0.08,
 310: 0.04,
 323: 0.04,
 432: 0.04,
 15: 0.04,
 233: 0.04,
 514: 0.04,
 32: 0.08,
 332: 0.08,
 437: 0.04,
 331: 0.04,
 100: 0.04,
 69: 0.04,
 196: 0.04,
 442: 0.04,
 1100: 0.041666666666666664,
 677: 0.041666666666666664,
 1081: 0.041666666666666664,
 1440: 0.041666666666666664,
 846: 0.041666666666666664,
 778: 0.041666666666666664,
 1492: 0.041666666666666664,
 900: 0.041666666666666664,
 909: 0.041666666666666664,
 735: 0.041666666666666664,
 1070: 0.041666666666666664,
 995: 0.041666666666666664,
 1027: 0.041666666666666664,
 806: 0.041666666666666664,
 1066: 0.08333333333333333,
 555: 0.041666666666666664,
 724: 0.041666666666666664,
 1078: 0.041666666666666664,
 795: 0.041666666666666664,
 1466: 0.041666666666666664,
 857: 0.041666666666666664,
 717: 0.041666666666666664,
 1253: 0.041666666666666664,
 2356: 0.043478260869565216,
 1919: 0.043478260869565216,
 2210: 0.043478260869565216,
 1587: 0.043478260869565216,
 2221: 0.043478260869

#### Vectorize `X_cat`

Convert you `X_cat` matrix into a numerical representation using `DictVectorizer`: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html. You will set the `sparse` argument to False. Call the result `X`.

In [253]:
# Write your code here
...
v = DictVectorizer(sparse=False)

X = v.fit_transform(X_cat)


In [254]:
X.shape

(300000, 2583)

In [255]:
X[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

## Programming: Building $\mathbf{y}$

You will now convert the list of language symbols into a $\mathbf{y}$ vector

Extract the language symbols from `dataset_small_feat` and call the resulting list `y_cat`

In [270]:
# Write your code here
y_cat = []
for i in range(len(dataset)):
    y_cat += [dataset[i][1]]


In [272]:
y_cat[:5]

['eng', 'cmn', 'dan', 'dan', 'eng']

Extract the set of language symbols and name it `y_symbols`. Then build two indices mapping the symbols to integers and the integers to symbols. Both indices will be dictionaries that you will call: `lang2idx`and `idx2lang`. Such a conversion is not necessary with sklearn. We do it because many other many machine-learning toolkits (keras or pytorch) require a numerical $\mathbf{y}$ vector and to learn how to carry out this conversion.

In [273]:
# Write your code here
y_symbols = set(y_cat)
lang2idx = {lang: i for i, lang in enumerate(y_symbols)}
idx2lang = {i: lang for i, lang in enumerate(y_symbols)}

In [274]:
idx2lang

{0: 'jpn', 1: 'swe', 2: 'fra', 3: 'eng', 4: 'dan', 5: 'cmn'}

In [276]:
lang2idx

{'jpn': 0, 'swe': 1, 'fra': 2, 'eng': 3, 'dan': 4, 'cmn': 5}

Convert your `y_cat` vector into a numerical vector. Call this vector `y`.

In [279]:
# Write your code here
y = [lang2idx[lang] for lang in y_cat]

In [280]:
y[:5]

[3, 5, 4, 4, 3]

## Programming: Building the Model

Create a neural network using sklearn with a hidden layer of 50 nodes and a relu activation layer: https://scikit-learn.org/stable/modules/neural_networks_supervised.html. Set the maximal number of iterations to 5, in the beginning, and verbose to True. Use the default values for the rest. You will call your classifier `clf`

In [298]:
# Write your code here
clf = MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, random_state=1234, verbose=True)

In [299]:
clf

MLPClassifier(hidden_layer_sizes=(50,), max_iter=10, random_state=1234,
              verbose=True)

### Training and Validation Sets

You will now split the dataset into a training and validation sets

#### We split the dataset
We use a training set of 80% and a validation set of 20%

In [300]:
training_examples = int(X.shape[0] * 0.8)

X_train = X[:training_examples, :]
y_train = y[:training_examples]

X_val = X[training_examples:, :]
y_val = y[training_examples:]

### Fitting the model

Fit the model on the training set

In [301]:
# Write your code here
model = clf.fit(X_train, y_train)

Iteration 1, loss = 0.29771445
Iteration 2, loss = 0.04065779
Iteration 3, loss = 0.02849672
Iteration 4, loss = 0.02339434
Iteration 5, loss = 0.02039174
Iteration 6, loss = 0.01842953
Iteration 7, loss = 0.01698608
Iteration 8, loss = 0.01582906
Iteration 9, loss = 0.01486970
Iteration 10, loss = 0.01425622


/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


## Predicting

Predict the `X_val` languages. You will call the result `y_val_pred`

In [302]:
# Write your code here
y_val_pred = clf.predict(X_val)

In [303]:
y_val_pred[:20]

array([0, 2, 5, 4, 4, 2, 1, 2, 5, 4, 1, 0, 5, 0, 4, 2, 3, 1, 0, 1])

In [304]:
y_val[:20]

[0, 2, 5, 4, 4, 2, 1, 2, 5, 4, 1, 0, 5, 0, 4, 2, 3, 1, 0, 1]

#### Evaluating

Use the `accuracy_score()` function to evaluate your model on the validation set

In [305]:
# evaluate the model
accuracy_score(y_val, y_val_pred)

0.9923

In [306]:
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

              precision    recall  f1-score   support

         jpn       1.00      1.00      1.00     10046
         swe       0.99      0.98      0.98      9905
         fra       1.00      1.00      1.00      9964
         eng       1.00      1.00      1.00     10043
         dan       0.98      0.99      0.98      9864
         cmn       1.00      1.00      1.00     10178

    accuracy                           0.99     60000
   macro avg       0.99      0.99      0.99     60000
weighted avg       0.99      0.99      0.99     60000

Micro F1: 0.9923
Macro F1 0.9922400771568108


### Confusion Matrix

In [307]:
confusion_matrix(y_val, y_val_pred)

array([[10018,     0,     0,     0,     0,    28],
       [    0,  9680,     9,     5,   211,     0],
       [    0,     3,  9944,    11,     6,     0],
       [    0,    13,    15, 10009,     6,     0],
       [    0,   115,     1,    16,  9729,     3],
       [    9,     2,     0,     5,     4, 10158]])

You may try to increase the number of iterations to improve the score. You may also try change the parameters of the multilayer percetron.

## Predict the language of a text

Now you will predict the languages of the strings below.

In [308]:
docs = ["Salut les gars !", "Hejsan grabbar!", "Hello guys!", "Hejsan tjejer!"]

In [309]:
build_freq_dict('Salut les gars !')

{56: 0.0625,
 234: 0.125,
 15: 0.125,
 69: 0.0625,
 432: 0.0625,
 86: 0.1875,
 32: 0.0625,
 331: 0.125,
 31: 0.0625,
 437: 0.0625,
 333: 0.0625,
 1209: 0.06666666666666667,
 640: 0.06666666666666667,
 582: 0.06666666666666667,
 1542: 0.06666666666666667,
 1492: 0.06666666666666667,
 900: 0.06666666666666667,
 739: 0.06666666666666667,
 1319: 0.06666666666666667,
 1238: 0.13333333333333333,
 982: 0.06666666666666667,
 1415: 0.06666666666666667,
 557: 0.06666666666666667,
 1161: 0.06666666666666667,
 1020: 0.06666666666666667,
 2362: 0.07142857142857142,
 1608: 0.07142857142857142,
 2199: 0.07142857142857142,
 2349: 0.07142857142857142,
 2284: 0.07142857142857142,
 1958: 0.07142857142857142,
 1720: 0.07142857142857142,
 1925: 0.07142857142857142,
 2370: 0.07142857142857142,
 2546: 0.07142857142857142,
 1752: 0.07142857142857142,
 1805: 0.07142857142857142,
 1670: 0.07142857142857142,
 2269: 0.07142857142857142}

Create features vectors from this list. Call this matrix `X_test`

In [370]:
# Write your code here
X_tester = []
for i in range(len(docs)):
    X_tester += [build_freq_dict(docs[i])]

t = DictVectorizer(sparse=False)
X_test = t.fit_transform(X_tester)


In [371]:
X_test

array([[0.125     , 0.        , 0.0625    , 0.0625    , 0.0625    ,
        0.0625    , 0.1875    , 0.        , 0.        , 0.125     ,
        0.        , 0.        , 0.125     , 0.0625    , 0.        ,
        0.0625    , 0.0625    , 0.06666667, 0.        , 0.06666667,
        0.        , 0.        , 0.        , 0.06666667, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.06666667, 0.        , 0.        , 0.        , 0.06666667,
        0.06666667, 0.06666667, 0.        , 0.        , 0.        ,
        0.        , 0.06666667, 0.        , 0.06666667, 0.        ,
        0.        , 0.13333333, 0.        , 0.06666667, 0.        ,
        0.06666667, 0.06666667, 0.        , 0.        , 0.06666667,
        0.        , 0.        , 0.07142857, 0.07142857, 0.        ,
        0.        , 0.        , 0.07142857, 0.07142857, 0.        ,
        0.        , 0.        , 0.07142857, 0.        , 0.        ,
        0.        , 0.        , 0.07142857, 0.  

And run the prediction that you will store in a variable called `pred_languages`

In [363]:
# Write your code here
pred_languages = clf.predict(X_test)

ValueError: X has 96 features, but MLPClassifier is expecting 2583 features as input.

In [318]:
pred_languages

array([2, 1, 3, 4])

## Building the Model with PyTorch
You will now recreate a PyTorch model with the same architecture as in sklearn.

### The Model
Create a model identical to the one you created with sklearn. Use the same activation function for the hidden layer and no activation in the last layer.

In [73]:
len(langs)

6

In [74]:
# Write your code here
class Model(nn.Module):
    def __init__(self, input_dim):
        ...

In [75]:
input_dim = X.shape[1]
model = Model(input_dim)
model

Model(
  (fc1): Linear(in_features=2583, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

Write the loss `loss_fn` and optimizer `optimizer`. As optimizer, use the same as in sklearn. See here: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [77]:
# Write your code here. (The solution is given)
loss_fn = nn.CrossEntropyLoss()    # cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

### The data loader

We convert the data to tensors

In [78]:
X_train = torch.Tensor(X_train)
y_train = torch.LongTensor(y_train)

X_val = torch.Tensor(X_val)
y_val = torch.LongTensor(y_val)

X_test = torch.Tensor(X_test)

In [79]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_train, y_train)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [80]:
model.train()

Model(
  (fc1): Linear(in_features=2583, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

Fit your network on your training set. Write a code similar to that seen during the lecture and use five epochs to start with.

In [81]:
# Write your code here
for epoch in range(5):
    loss_train = 0
    for X_batch, y_batch in dataloader:
        ...
    print(loss_train)

765.704064067424
174.51052177065867
140.88319964771836
122.14930617929895
108.1038764459961


Predict the validation set `X_val` in the form of logits. Call the result: `Y_val_pred_logits`

In [82]:
model.eval()

Model(
  (fc1): Linear(in_features=2583, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=6, bias=True)
)

In [83]:
# Write your code here
Y_val_pred_logits = ...

In [84]:
Y_val_pred_logits[:5]

tensor([[ -5.9929,   6.8918,  -3.0228,  -9.0067,  -4.4374,  -9.8311],
        [ -5.7582, -10.4759, -12.6710,  -3.3666,  18.9000, -12.0623],
        [ -6.9647,  -9.3725, -22.0371,  22.4224,  -8.4151, -14.8630],
        [ -8.7401, -11.8870,  -8.4524,  -2.2943,  16.8711,  -7.5894],
        [ -3.7173,  -5.8539,  -5.3409,  -9.7889,  -6.0550,  11.3400]],
       grad_fn=<SliceBackward0>)

Predict the validation set `X_val` in the form of probabilities. Use `torch.softmax()` for that and call the result: `Y_val_pred_proba`

In [85]:
# Write your code here
Y_val_pred_proba = ...

In [86]:
Y_val_pred_proba[:4]

tensor([[2.5364e-06, 9.9994e-01, 4.9445e-05, 1.2455e-07, 1.2017e-05, 5.4616e-08],
        [1.9547e-11, 1.7467e-13, 1.9448e-14, 2.1365e-10, 1.0000e+00, 3.5748e-14],
        [1.7271e-13, 1.5547e-14, 4.9143e-20, 1.0000e+00, 4.0500e-14, 6.4144e-17],
        [7.5369e-12, 3.2398e-13, 1.0050e-11, 4.7487e-09, 1.0000e+00, 2.3820e-11]],
       grad_fn=<SliceBackward0>)

Extract the categories from the probabilities in `Y_val_pred_proba`. Use the `torch.argmax()` function. Call the result `y_val_pred`. Check that the prediction corresponds to the real values.

In [87]:
# Write your code here
y_val_pred = ...

In [88]:
y_val_pred[:20]

tensor([1, 4, 3, 4, 5, 1, 2, 0, 2, 3, 4, 1, 0, 5, 0, 0, 5, 5, 0, 0])

In [89]:
y_val[:20]

tensor([1, 4, 3, 4, 5, 1, 2, 0, 2, 3, 4, 1, 0, 5, 0, 0, 5, 5, 0, 0])

Print the evaluation

In [90]:
print(classification_report(y_val, y_val_pred, target_names=y_symbols))
print('Micro F1:', f1_score(y_val, y_val_pred, average='micro'))
print('Macro F1', f1_score(y_val, y_val_pred, average='macro'))

              precision    recall  f1-score   support

         dan       0.98      0.98      0.98     10081
         swe       0.98      0.98      0.98     10001
         eng       0.99      1.00      1.00      9989
         jpn       1.00      1.00      1.00      9868
         cmn       1.00      1.00      1.00     10027
         fra       1.00      1.00      1.00     10034

    accuracy                           0.99     60000
   macro avg       0.99      0.99      0.99     60000
weighted avg       0.99      0.99      0.99     60000

Micro F1: 0.9922333333333333
Macro F1 0.9922547719497999


Print the confusion matrix

In [91]:
confusion_matrix(y_val, y_val_pred)

array([[ 9914,   146,    20,     0,     0,     1],
       [  167,  9815,    15,     0,     1,     3],
       [   15,     6,  9962,     0,     0,     6],
       [    0,     0,     0,  9839,    29,     0],
       [    3,     2,     1,    14, 10005,     2],
       [    6,     5,    24,     0,     0,  9999]])

Predict your languages with PyTorch. Reuse `X_test` and call the result `Y_test_pred_proba`.

In [92]:
# Write your code here
Y_test_pred_proba = ...

In [93]:
Y_test_pred_proba

tensor([[1.8276e-07, 6.6974e-07, 1.4123e-07, 1.6559e-09, 2.8414e-08, 1.0000e+00],
        [4.1081e-04, 9.9952e-01, 2.4307e-05, 9.6759e-07, 4.6090e-05, 1.8566e-06],
        [3.9380e-02, 4.1912e-05, 9.4139e-01, 7.8594e-06, 8.5014e-04, 1.8331e-02],
        [9.9818e-01, 1.8032e-03, 7.3723e-08, 9.3494e-07, 1.2838e-05, 9.8474e-07]],
       grad_fn=<SoftmaxBackward0>)

From the probabilities, extract the predicted languages and map them to strings. Call the results `pred_languages_pytorch`.

In [94]:
# Write your code here
pred_languages_pytorch = ...

In [95]:
pred_languages_pytorch

['fra', 'swe', 'eng', 'dan']

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. Do not forget to:
   * Summarize CLD3 and outline its architecture
   * Identify the features used by CLD3
   * Describe your architecture and tell how it is different from CLD3
   * Include the feature matrix you computed manually
   * Outline the differences between sklearn and PyTorch

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is October 6, 2023.

## Postscript from Pierre Nugues

I created this assignment from an examination I wrote in 2019 for the course on applied machine learning. I simplified it from the `README.md` on GitHub, https://github.com/google/cld3. I found the C++ code difficult to understand and I reimplemented a Keras/Tensorflow version of it from this `README`. Should you be interested, you can find it here: https://github.com/pnugues/language-detector.